In [1]:
import numpy as np
import torch.nn as nn 
from torchsummary import summary 


### Loading Data

In [ ]:
data = np.load('processed_eeg_data.npz')
X_train = data['X_train']
X_val = data['X_val']
y_train = data['y_train']
y_val = data['y_val']

In [ ]:
class CNNLSTM(nn.Module):
# Input size is 8282*23*256 num samples x channels x time steps 
    def __init__(self, ):
        super(CNNLSTM, self).__init__()
        self.conv1 = nn.Conv1d(23, 16, 3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, 3, padding=1)
        self.lstm1 = nn.LSTM(32, 64, num_layers=2, batch_first=True)
        self.lstm2 = nn.LSTM(64, 64, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.ReLU()(self.conv2(x))
        x = x.permute(0, 2, 1)

        x, _ = self.lstm1(x)
        x = nn.dropout(x, p=0.5)
        x, _ = self.lstm2(x)
        x = nn.dropout(x, p=0.5)
        x = x[:, -1, :]  # Take the last time step

        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x
